In [ ]:
import corner
import emcee
import time
import numpy as np

from joblib import load
from pathlib import Path
from multiprocessing import Pool
from scipy.optimize import minimize
from IPython.display import display, Math
from tensorflow.keras.models import load_model

from modules.network import r_squared

In [ ]:
cwd = Path.cwd()
result_path = cwd /'results'
data_file_path = cwd /'data'

xd = np.load(data_file_path / 'x_true.npy', allow_pickle=True)
yd = np.load(data_file_path / 'y_true.npy', allow_pickle=True)
yderr = np.load(data_file_path / 'yerr.npy', allow_pickle=True)

model = load_model(result_path / 'ANN_model.h5', custom_objects={'r_squared': r_squared})
# Load the saved scaler
flux_scaler = load(data_file_path / 'flux_scaler.joblib')

In [ ]:
def Model(x, m, b):
    x_grid = np.linspace(-10, 10, 100)
    params = [m, b]
    y_pred = model.predict([params], verbose=0)
    y_pred_d = flux_scaler.inverse_transform(y_pred)

    return np.interp(x, x_grid, y_pred_d[0])

def log_likelihood(theta):
    m, b, log_f = theta
    model = Model(xd, m, b)
    sigma2 = yderr**2 + model**2 * np.exp(2 * log_f)
#    sigma2 = yerr**2 + np.exp(2 * log_f)
    return -0.5 * np.sum((yd - model) ** 2 / sigma2 + np.log(sigma2))

def log_prior(theta):
    m, b, log_f = theta
    if (-5 < m < 5) and (-10 < b < 10) and (-10.0 < log_f < 1.0):
        return 0.0
    return -np.inf

def log_probability(theta):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta)

In [ ]:
np.random.seed(42)
# The "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

nll = lambda *args: -log_likelihood(*args)
initial = np.array([m_true, b_true, np.log(f_true)]) + 0.1 * np.random.randn(3)
soln = minimize(nll, initial)
m_ml, b_ml, log_f_ml = soln.x

print("Maximum likelihood estimates:")
print("m = {0:.3f}".format(m_ml))
print("b = {0:.3f}".format(b_ml))
print("f = {0:.3f}".format(np.exp(log_f_ml)))

plt.errorbar(xd, yd, yerr=yderr, fmt=".c", capsize=0)
plt.plot(xd, m_true * xd + b_true, "c", alpha=0.3, lw=3, label="truth")
plt.plot(xd, Model(xd, m_ml, b_ml), ":c", label="ML")
plt.legend(fontsize=14)
plt.xlabel("x")
plt.ylabel("y")

In [ ]:
params = soln.x[:2]
guess_par = params

init = soln.x

pos = init + 1e-2 * np.random.randn(48, len(init))
nwalkers, ndim = pos.shape

In [ ]:
start_time = time.time()

print('start sampler')

'''with Pool() as pool:
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, log_probability, pool=pool
    )

    sampler.run_mcmc(pos, 1000, progress=True)
'''

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability)
sampler.run_mcmc(pos, 1000, progress=True)


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["m", "b",'logf']
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "c", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
plt.show()

In [ ]:
tau = sampler.get_autocorr_time()
print(tau)

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
print(flat_samples.shape)


for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))

In [ ]:
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(
    flat_samples, labels=labels, truths=[m_true, b_true, np.log(f_true)],
)